In [1]:
import pandas as pd
pd.set_option('display.max_rows', None) # Print all rows

In [2]:
# FIX SSL: CERTIFICATE_VERIFY_FAILED with Python3
# https://stackoverflow.com/a/49174340
import ssl

ssl._create_default_https_context = ssl._create_unverified_context

In [3]:
url = 'https://www.juntadeandalucia.es/institutodeestadisticaycartografia/badea/stpivot/stpivot/Print?cube=013d884c-cfc5-4c21-9e78-e26225682afd&type=3&foto=si&ejecutaDesde=&codConsulta=38668&consTipoVisua=JP'
fields = ['Lugar de residencia', 'Medida', 'Valor']
df = pd.read_csv(url, delimiter=';', usecols=fields, decimal=',')

In [4]:
df[df['Lugar de residencia'] == 'Huelva']

,Lugar de residencia,Medida,Valor
0,Huelva,Población media anual,513170
1,Huelva,Confirmados PDIA,61776
2,Huelva,Confirmados PDIA 14 días,7233
3,Huelva,Tasa PDIA 14 días,"1409,4744431669817"
4,Huelva,Confirmados PDIA 7 días,2907
5,Huelva,Tasa PDIA 7 dias,"566,4789445992556"
6,Huelva,Total Confirmados,61996
7,Huelva,Curados,49679
8,Huelva,Fallecidos,429


In [5]:
df['Valor'] = pd.to_numeric(df['Valor'],errors='coerce')

In [6]:
dfAnterior = pd.read_csv('covid_huelva.csv', index_col='Lugar de residencia')

In [7]:
# Confirmados PCR provincia Huelva
p = df[df['Lugar de residencia'] == 'Huelva'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]
print('Confirmados PCR en la provincia de Huelva: ', p)

Confirmados PCR en la provincia de Huelva:  61776.0


/tmp/ipykernel_34331/1396922249.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  p = df[df['Lugar de residencia'] == 'Huelva'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]


In [8]:
# Confirmados PCR Huelva capitalpd
c = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]
print('Confirmados PCR en Huelva capital: ', c)

Confirmados PCR en Huelva capital:  17857.0


/tmp/ipykernel_34331/2509482988.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  c = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]


In [9]:
c7 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 7 días']['Valor'].values[0]
c14 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 14 días']['Valor'].values[0]
p = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Población media anual']['Valor'].values[0]

/tmp/ipykernel_34331/242626003.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  c7 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 7 días']['Valor'].values[0]
/tmp/ipykernel_34331/242626003.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  c14 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 14 días']['Valor'].values[0]
/tmp/ipykernel_34331/242626003.py:3: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  p = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Población media anual']['Valor'].values[0]


In [10]:
print('Huelva (capital)\n')
#print('Confirmados PCR 7 días x 1,000 habitantes: ', f"{c7/(p/1000):.5f}")
print('Un positivo PCR cada', f"{p/c7:.0f}", 'personas en los últimos 7 días', '\n')
#print('Confirmados PCR 14 días x 1,000 habitantes: ', f"{c14/(p/1000):.5f}")
print('Un positivo PCR cada', f"{p/c14:.0f}", 'personas en los últimos 14 días')

Huelva (capital)

Un positivo PCR cada 144 personas en los últimos 7 días 

Un positivo PCR cada 61 personas en los últimos 14 días


In [11]:
filtro = ((df['Medida'] == 'Confirmados PDIA')| (df['Medida'] == 'Confirmados PDIA 7 días') | (df['Medida'] == 'Población media anual' ) | (df['Medida'] == 'Confirmados PDIA 14 días')) & (df['Valor'] > 0)
tabla = pd.pivot_table(df.loc[filtro], values='Valor', index='Lugar de residencia', columns='Medida').dropna()

In [12]:
# Rename columns
tabla.columns = ['Confirmados PDIA', 'Confirmados PDIA 14 días', 'Confirmados PDIA 7 días', 'Población']

In [13]:
tabla['tasa7'] = tabla['Confirmados PDIA 7 días']/tabla['Población']*100000
tabla['tasa14'] = tabla['Confirmados PDIA 14 días']/tabla['Población']*100000

In [14]:
tabla['Diferencia PDIA'] = tabla['Confirmados PDIA'] - dfAnterior['Confirmados PDIA']

In [15]:
# Nuevos casos desde la última captura
tabla[tabla['Diferencia PDIA'] != 0].sort_values('Diferencia PDIA', ascending=False)

,Confirmados PDIA,Confirmados PDIA 14 días,Confirmados PDIA 7 días,Población,tasa7,tasa14,Diferencia PDIA
Lugar de residencia,,,,,,,
Huelva,61776.0,7233.0,2907.0,513170.0,566.478945,1409.474443,673.0
Huelva-Costa,36269.0,4076.0,1680.0,289548.0,580.214679,1407.711329,287.0
Sierra de Huelva-Andévalo Central,6724.0,1317.0,554.0,67391.0,822.068229,1954.266890,230.0
Huelva (capital),17857.0,2352.0,997.0,143837.0,693.145714,1635.184271,175.0
Condado-Campiña,18330.0,1825.0,666.0,156231.0,426.291837,1168.142046,152.0
Moguer,2571.0,240.0,110.0,21867.0,503.041112,1097.544245,40.0
Valverde del Camino,1477.0,285.0,99.0,12750.0,776.470588,2235.294118,32.0
Aljaraque,2396.0,314.0,141.0,21474.0,656.607991,1462.233399,30.0
Nerva,404.0,91.0,50.0,5169.0,967.305088,1760.495260,21.0


In [16]:
if tabla[tabla['Diferencia PDIA'] != 0]['Diferencia PDIA'].sum() != 0:
  # Save to csv
  tabla.to_csv('covid_huelva.csv', encoding='utf-8', header=True, index=True)  

In [17]:
tabla.sort_values('tasa14', ascending=False)

,Confirmados PDIA,Confirmados PDIA 14 días,Confirmados PDIA 7 días,Población,tasa7,tasa14,Diferencia PDIA
Lugar de residencia,,,,,,,
Hinojales,22.0,20.0,17.0,330.0,5151.515152,6060.606061,4.0
Rosal de la Frontera,162.0,70.0,36.0,1697.0,2121.390689,4124.926341,10.0
Campofrío,92.0,24.0,5.0,713.0,701.262272,3366.058906,2.0
Jabugo,176.0,75.0,27.0,2260.0,1194.690265,3318.584071,11.0
Villanueva de las Cruces,41.0,10.0,6.0,387.0,1550.387597,2583.979328,0.0
Cerro de Andévalo (El),285.0,59.0,31.0,2327.0,1332.187366,2535.453373,19.0
Aroche,244.0,77.0,28.0,3054.0,916.830386,2521.283563,13.0
Zalamea la Real,223.0,70.0,37.0,3054.0,1211.525868,2292.075966,15.0
Valverde del Camino,1477.0,285.0,99.0,12750.0,776.470588,2235.294118,32.0


In [18]:
# Peso de la tasa de la última semana en relación con la tasa a 14 días 
# Mejor cuanto menor. Valor 1 significa que no había casos antes de 7 días.
tabla['7/14'] = tabla['tasa7'] / tabla['tasa14']
tabla.fillna(0).sort_values('7/14', ascending=True)

,Confirmados PDIA,Confirmados PDIA 14 días,Confirmados PDIA 7 días,Población,tasa7,tasa14,Diferencia PDIA,7/14
Lugar de residencia,,,,,,,,
San Silvestre de Guzmán,79.0,12.0,1.0,614.0,162.866450,1954.397394,0.0,0.083333
Manzanilla,224.0,36.0,5.0,2118.0,236.071766,1699.716714,2.0,0.138889
Escacena del Campo,309.0,25.0,4.0,2287.0,174.901618,1093.135111,0.0,0.160000
Villarrasa,275.0,17.0,3.0,2211.0,135.685210,768.882858,2.0,0.176471
Villablanca,339.0,30.0,6.0,2885.0,207.972270,1039.861352,2.0,0.200000
Campofrío,92.0,24.0,5.0,713.0,701.262272,3366.058906,2.0,0.208333
Lucena del Puerto,448.0,37.0,8.0,3261.0,245.323520,1134.621282,1.0,0.216216
Almendro (El),77.0,8.0,2.0,840.0,238.095238,952.380952,1.0,0.250000
Puerto Moral,27.0,4.0,1.0,278.0,359.712230,1438.848921,0.0,0.250000
